In [1]:
import json
import pandas as pd
from sql_queries import ValorantQueries

In [2]:
vq = ValorantQueries()
vq.set_match(10597)

Connected to database valorant successfully
[('teams',), ('players',), ('events',), ('series',), ('matches',), ('rounds',), ('roundeconomies',), ('roundevents',), ('roundlocations',), ('playermapinstance',), ('assists',), ('maps',)]


In [12]:
loadout_df = vq.get_full_loadout_table()
round_amount = loadout_df["Round"].max()
split_aux = {i: [] for i in range(1, round_amount + 1)}
for row in loadout_df.iterrows():
    content = row[1]
    round_id = row[1].Round
    split_aux[round_id].append(content)

group_aux = {}
for key, value in split_aux.items():
    raw_group = pd.DataFrame(value)
    compact = raw_group.groupby(['Team ID']).agg({'Remaining Creds': 'sum', 'Loadout Value': 'sum',
                                                  'Shield': 'sum', 'Shield Price': 'sum',
                                                  'Weapon Price': 'sum'})
    compact["Round"] = [key] * 2
    compact["Team ID"] = compact.index
    clean_dict = {"Round": list(compact["Round"]),
                    "Team ID": list(compact["Team ID"]),
                    "Remaining Creds": list(compact["Remaining Creds"]),
                    "Loadout Value": list(compact["Loadout Value"]),
                    "Shield Amount": list(compact["Shield"]),
                    "Shield Price": list(compact["Shield Price"]),
                    "Weapon Price": list(compact["Weapon Price"])}
    compact = pd.DataFrame(clean_dict)
    group_aux[key] = compact

final_df = pd.concat(list(group_aux.values()), axis=0, ignore_index=True)
final_df["Economy"] = final_df["Remaining Creds"] + final_df["Loadout Value"]
final_df

,Round,Team ID,Remaining Creds,Loadout Value,Shield Amount,Shield Price,Weapon Price,Economy
0,1,78,0,3700,50,800,2000,3700
1,1,752,100,3800,0,0,2500,3900
2,2,78,3600,14300,250,5000,5800,17900
3,2,752,8600,2600,0,0,500,11200
4,3,78,13300,14400,175,3400,7400,27700
5,3,752,400,22100,250,5000,14500,22500
6,4,78,1700,23400,225,4400,16600,25100
7,4,752,7600,22000,250,5000,14500,29600
8,5,78,9200,5600,50,800,3500,14800
9,5,752,11200,24200,250,5000,16600,35400


In [7]:
loadout_df

,Match ID,Round ID,Round,Player ID,Team ID,Remaining Creds,Loadout Value,Agent ID,Weapon ID,Armor ID,Shield,Shield Price,Weapon Name,Weapon Price,Utility Value
0,10597,172631,1,1408,78,0,500,3,12,0,0,0,Ghost,500,0
1,10597,172631,1,2062,78,0,800,4,11,1,25,400,Classic,0,400
2,10597,172631,1,3770,752,0,600,3,10,0,0,0,Frenzy,500,100
3,10597,172631,1,919,752,100,700,11,12,0,0,0,Ghost,500,200
4,10597,172631,1,2476,752,0,800,4,12,0,0,0,Ghost,500,300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215,10597,172652,22,102,78,7400,4400,11,4,2,50,1000,Vandal,2900,500
216,10597,172652,22,3769,752,0,2800,10,19,2,50,1000,Stinger,1100,700
217,10597,172652,22,639,78,1800,4400,12,4,2,50,1000,Vandal,2900,500
218,10597,172652,22,330,752,100,2400,12,19,2,50,1000,Stinger,1100,300


### AAA

In [3]:
loadouts_df = vq.get_full_loadout_table()
side_info = vq.get_side_info()
team_a: int = side_info["team_a"]
team_b: int = side_info["team_b"]
attacking_first = side_info["attacking_first"]
defending_first = side_info["defending_first"]
events = vq.get_events()
gamestate_df = pd.DataFrame(columns=['MatchID', 'MapName', 'ATK_Guns', 'ATK_Shields'])
trimmed_events = [e[:12] for e in events]
events_raw_df = pd.DataFrame(trimmed_events, columns=["ta", "round_event_id", "round_id", "round_number",
                                              "round_time_millis", "player_id", "victim_id", "event_type",
                                              "damage_type", "weapon_id", "ability", "attacking_team_number"])

ValueError: Length of values (2) does not match length of index (1)